*TAP US30*

STREAMING

CLIENT PHP

Il primo passo della pipeline è quello di creare un client che s'interfacci
con l'API di Polygon e faccia richiesta dei dati, considerando la sola giornata 
precedente. Quest'ultimo sarà mandato in esecuzione su un container.Di seguito, 
vediamo come ottenere l'opportuno Dockerfile, da cui potremo creare l'immagine 
che, nel nostro "docker-compose", chiameremo "producer".

In [ ]:
FROM alpine
RUN apk update && apk add php composer php-fileinfo
WORKDIR /app
COPY batch_extract.php /app
RUN composer require polygon-io/api -W
CMD ["php", "batch_extract.php"]

L'immagine base è quella della versione "alpine" di Linux, scelta in quanto
tra le più "leggere" in termini di spazio sul disco. Il gestore di pacchetti
si chiama "apk", da cui installiamo le librerie php e composer. La directory "/app" nel
container è quella in cui copiamo il codice "batch_extract.php", il cui
contenuto descriveremo in seguito. Eseguiamo il comando "composer" per richiedere
la libreria da utilizzare nel nostro codice php così da accedere all'API.
Dopodichè, una volta partito il container, viene eseguto il comando che
innesca l'esecuzione del suddetto codice.

*batch_extract.php*

Innanzitutto, salviamo la API key in una opportuna variabile. Creaiamo un array
associativo con come chiave il nome dell'azienda e come valore il settore in cui
opera. Dichiariamo poi un oggetto di tipo "Rest" sul quale chiameremo una serie di
funzioni per prelevare i dati che ci occorrono. Il più importante è il metodo "get"
che richiede come parametri il nome dell'azienda, un intero, la data di inizio e 
quella di fine entro cui prelevare le informazioni e l'unità di tempo, che indichiamo
come il singolo giorno. In un ciclo for avanzato, scorriamo l'array associativo. ogni
12 secondi, inviamo una richiesta attraverso l'API. Salviamo queste informazioni su una
variabile che viene passata alla nostra funzione "write_batch", insieme al nome 
dell'azienda e alla categoria a cui appartiene, aggiungendo poi anche l'array
associativo stesso. 
Dopo aver proceduto, alla creazione della cartella e del file, si salva in una variabile
l'array alla posizione "results" dell'array ottenuto dalla get. Si effettua un ciclo su
tutte le n-uple che caratterizzano i dati della specifica azienda, a partire dall'array
salvato nella variabile e, dopo aver aggiunto il ticker symbol in questione (dato che 
altrimenti non sarebbe presente per ogni n-upla, il che creerebbe dei problemi per il 
successivo processing), si scrive sul file appena creato il contenuto dell'array, 
sottoforma di json. Fuori dal ciclo, si procede alla chiusura del file.
Quello che otterremo è quindi la presenza di 6 cartelle, una per ogni settore a cui le
30 aziende appartengono, e ognuna di queste cartelle avrà dei file con estensione "txt" 
composti da n j-son, uno per ogni giorno in cui ogni azienda opera nel mercato azionario.

*DATA INGESTION*

Innanzitutto, abbiamo utilizzato Fluentd come framework. Esso permette 
con facilità di accedere a files salvati nel proprio file system e usarli 
quali source.
Dockerfile.

In [ ]:
FROM fluentd
USER root
RUN apk add ruby-dev
RUN gem install fluent-plugin-kafka --no-doc

Partiamo dall'immagine "vergine" di fluentd, scaricata direttamente dalla
repository pubblica, per poi installare ruby e gem, quest'ultimo il gestore
di pacchetti opportuno per installare il plugin che permette lo scambio di
dati tra fluentd e kafka. L'immagine risultante sarà chiamata "fluentkafka".

*fluent.conf*

La tappa più importante, dopo aver creato opportunemente il
Dockerfile e dunque aver avuto a disposizione l'immagine per il container,
è quella di scrivere correttamente il file di configurazione. 
Esistono due tag fondamentali che lo caratterizzano, ovvero "<source>" e 
"<match>", rispettivamente a indicare l'input e l'output della data 
ingestion. 
Source.

L'annotazione "@type tail" si utilizza per indicare che si intenda aprire
un file. Questi vengono letti, come suggerisce il nome stesso, dall'ultima
riga alla prima a meno che, come abbiamo fatto, non si setti la variabile
read_from_head a "true".  
I files da cui stiamo leggendo, come già spiegato nella sezione CLIENT PHP,
sono caratterizzati da un json per ogni riga. Dunque, indicheremo "format
json" affinchè vengano riconosciuti da fluentd come tali.
Visto che abbiamo scelto di suddividere le aziende in base al settore,
continuiamo a mantenere questi gruppi specificando una source per ognuno
di essi. Qualora non li taggassimo opportunemente, non riusciremmo ad 
utilizzarli in modo specifico all'interno del match. Per cui, per ognuno
di essi, scriveremo "tag" seguito dal nome del settore.
Infine, sarà fondamentale indicare il path da cui prelevarli, così come
un path temporaneo, su "pos_file", che non indicherà altro che la cartella
temporanea in cui si immagazzineranno i dati prima del loro invio in output.
Match.

L'annotazione "@type kafka2" indicherà che l'output verrà reindirizzato a 
kafka. Ciò è possibile in virtù dell'installazione del plugin che connette
fluentd a kafka, da Dockerfile. 
Il suddetto plugin ci permetterà di specificare due unità fondamentali per
kafka, ovvero i brokers e i topics. Abbiamo scelto, per leggibilità, di 
chiamare ogni broker come "k-[nomebroker]" e di bindarlo con la porta
"9092", la quale sarà mappata a una porta effettiva al di fuori del container
dell'istanza di kafka, che coincide con il broker stesso 
(si veda il docker_compose). Come nome del topic, avremo lo stesso nome
del tag specificato in "<source>". Avremo quindi un "<match>" per
ogni "<source>". Per specificare quali dati in input inoltreremo, il tag
sarà "<match [nometag]>".
Risulta necessario specificare anche il tipo di file in output, attraverso il
tag "@type json" all'interno del tag "<format>", rigorosamente annidato dentro
il match.

DATA DISTRIBUTION
Al fine di smistare opportunemente i dati acquisiti, abbiamo usato Apache Kafka. Le unità fondamentali tipiche di questo servizio sono i brokers, i topics, i producers e i consumers. La nostra scelta è stata di creare, come detto, 6 topic, forniti da altrettanti brokers. Ai fini della pipeline, non è necessario istruire producers e consumers (per debug, così da accertarci che i dati effettivamente fossero inviati da fluentd, abbiamo passato argomenti al file "sh" del consumer affinchè stampasse su console il contenuto di questi json)

Dockerfile

In [ ]:
FROM bitnami/kafka
RUN rm -f /bitnami/kafka/data/.lock
COPY init.sh /opt/bitnami/scripts/kafka
COPY init /opt/bitnami/scripts/kafka
CMD ["/opt/bitnami/scripts/kafka/init"]

Ognuna della 6 istanze di kafka, identificate da un broker per ciascuna, si basa su questa immagine. Dopo aver scaricato l'immagine ufficiale di kafka, si aggiunge lo script "init.sh" all'interno del file system del container, così come "init" (il file compilato basato sul codice in c che ci permette un'opportuna temporizzazione tra il deployment del server e la creazione dei topics, rigorosamente l'una dopo l'altra, eseguendo prima lo script run.sh e poi init.sh) e si esegue "init".
L'immagine risultante sarà chiamata "kafka_init".

Zookeeper e ZooNavigator

Per navigare agilmente tra l'amplia offerta di servizi di Kafka, è sempre necessario che con i server kafka si attivi anche un server zookeeper e un altro server, nel nostro caso "zoonavigator", che consenta l'accesso via browser dell'interfaccia grafica di zookeeper. 

Dockerfile

In [ ]:
FROM zookeeper
RUN /apache-zookeeper-3.9.1-bin/bin/zkCli.sh << "delete /brokers/ids/*"

Questo Dockerfile parte dall'immagine ufficiale di zookeeper. L'immagine che costituisce sarà chiamata "zookeeper_clean"

DATA PROCESSING (BATCH)

process.py

In [ ]:
spark = SparkSession.builder.master('local[*]').config("spark.driver.memory","15g")\
    .appName("tapus30").getOrCreate()
# spark = SparkSession(sc)
sc = spark.sparkContext
sc.setLogLevel("WARN")

names=["cgoods","financial","energy","health","industrial","tech"]
indexes=[Elasticsearch("http://es_cgoods:9200"),Elasticsearch("http://es_financial:9200"),Elasticsearch("http://es_energy:9200"),Elasticsearch("http://es_health:9200"),Elasticsearch("http://es_industrial:9200"),Elasticsearch("http://es_tech:9200")]
prediction_data=[]
historical_data=[]
day_in_ms = 86400000
window_size = 12

Innanzitutto, apriamo una sessione di Spark, scegliendo come master il nostro stesso container, garantendoci poi che renda disponibili un massimo di 15 gigabyte di ram per l'esecuzione del driver che stiamo scrivendo. Specifichiamo il nome del nostro driver e salviamo le impostazioni sull'oggetto "spark". Su questo verra richiamato "sparkContext" per l'apertura effettiva del context e salviamo ciò che ci viene restituito sulla variabile "sc", su cui chiameremo poi il metodo "setLogLevel", per decidere il tipo di logs che vogliamo siano prodotti all'esecuzione del codice.
Creiamo poi un array con i nomi dei settori, che saranno anche i nomi degli indici su elasticsearch. Dichiariamo un array di oggetti Elasticsearch attraverso cui, successivamente, potremo effettivamente inviare i dati alle sue sei diverse istanze, ancora una volta una per settore. 
Dichiariamo poi due array vuoti, che saranno popolati dai dati restituitici dal modello di machine learning che useremo (prediction_data) e dai dati che preleveremo dal nostro file system, relativi ai 2 anni di operazioni in borsa (historical_data). 
Impostiamo un offset fondamentale per operare con i timestamp (ovvero le date espresse in millisecondi, a partire dall'omonimo campo all'interno dei json), cioè quello che ci consente di aggiungere un giorno alla data da cui partiamo. Altra importante variabile è quella "window_size", che ci permetterà di eseguire la funzione ricorsiva che vedremo per implementare una sliding window di 12 giorni, base su cui il modello via via restituirà l'ultima previsione. 

In [ ]:
for name in names:
    df = spark.read.json("/data/"+name)
    assembler = VectorAssembler(inputCols=['open','high','low'],outputCol='features')
    output = assembler.transform(df).select('features','close','tickerSymbol','timestamp')
    lr = LinearRegression (featuresCol='features',labelCol='close',maxIter=10,regParam=0.3,elasticNetParam=0.7)
    trained_model = lr.fit(output)
    print("processing data for "+name+"...")
    # Performs the recursive prediction based on the trained_model that we have just created
    predictions = recursive_prediction(df, trained_model)
    print("...done")
    # If we have previously executed the batch process (this file), 
    # we have to overwrite the model with fresh data
    lr.write().overwrite().save("/models/"+name)
    historical_data.append(format_data(output,"close").toPandas().to_dict(orient="records"))
    prediction_data.append(format_data(predictions.withColumnRenamed("close","prediction"),"prediction").toPandas().to_dict(orient="records"))

Eseguiamo un ciclo for sull'array di nomi dei settori. In esso, carichiamo su "df" i files contenuti nella cartella del settore su cui ci troviamo. Costruiamo un oggetto di tipo "VectorAssembler" che non restituirà altro che uno schema, il quale condenserà i dati contenuti nei campi "open", "high" e "low" in un unico vettore, chiamato features, che si passa legittimamente al modello per il machine learning.
Passiamo questo assembler al modello, chiamando su di esso il metodo "transform" e passando il dataframe appena caricato come argomento. Selezioniamo, sull'oggetto risultante, le colonne "features", "close", "tickerSymbol" e "timestamp" e carichiamo questo oggetto sulla variabile "output".
Applichiamo l'algoritmo di linear regression a questo dafaframe "output", dapprima ottenendo "lr", che non sarà altro che un oggetto contenente le impostazioni dei parametri di cui ha bisogno la linear regression, (nello specifico, il contenuto del vettore appena creato sarà alla base della previsione, il cui valore sarà riportato nella colonna "close", ed eseguiremo un massimo di 10 iterazioni nell'allenamento). Poi salveremo il modello a seguito del suo training in "trained_model", ottenuto dalla chiamata del metodo "fit" su "lr", passando come parametro "output".
Abbiamo tutto ciò che ci occorre per chiamare la funzione "rucursive_prediction", a partire dal dataframe iniziale e dal modello appena allenato. La funzione ci restituirà il dataframe "prediction", che conterrà il valore "close" per gli 11 giorni previsti a partire dalla sliding window, che inzia dagli ultimi 12 giorni di dati storici. 
Per salvare il modello "lr" nell'apposita cartella contenente il model dello specifico settore. chiamiamo sulla sua variabile il metodo "write", poi "overwrite" e poi "save" con il path su cui salvarlo.
Popoliamo adesso quei due array vuoti specificati all'inizio del codice, che conterranno oggetti di tipo dictionary (i quali elasticsearch potrà accettare), che nel primo caso saranno costituiti dalle n_uple del settore specifico corrispondenti alla colonna "close", nell'altro saranno corrispondenti alla colonna "prediction". 

In [ ]:
for i in range(len(indexes)):
    print("sending index "+names[i]+" to elasticsearch...")
    save_and_send_data("prediction",prediction_data,names[i],indexes[i])
    save_and_send_data("historical",historical_data,names[i],indexes[i])

Per ognuno degli oggetti di tipo Elasticsearch nell'array indexes, inviamo la specifica di quale dei due indici, due per settore, vogliamo popolare, dell'oggetto di tipo dictionary, del nome del settore a cui è associato e del nome dell'oggetto elasticsearch a cui vogliamo inviare i dati.